# STEP 1: Convert raw data to modeling data

The raw data is a .czi file as returned by the zeiss zen software. We need to convert the raw data to data that is more appropriate for segmentation modeling purposes (like a small .tif or .jpg file).

Data conversion process includes:

0. (Before conversion) Place raw data in data/raw

1. Convert raw czi data to (.tif) images in data/interim/converted/images

2. Label converted images with MATLAB segmentation tool.

3. Crop 2048x2048 images to 4 1024x1024 images and downsize to 256x256

4. Save these downsized images in data/interim/cropped
    
5. Split data from data/interim/cropped into   

## Step 1.1: Convert raw czi data to .tif images

In [25]:
%load_ext autoreload
%autoreload 2
import os
from src.data.convert_czi import czi_convert_directories

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Set the path to the input and output data directories and the desired converted image extension.

In [26]:
input_dir = "../data/raw"
output_dir = "../data/interim/converted/images"
ext = '.tif'

Run the conversion script to convert all czi files in sub-directories of the input_dir. Save the files in a mirrored directory structure located in the output_dir.

In [27]:
czi_convert_directories(input_dir=input_dir, output_dir=output_dir, overwrite=False, ext=".tif")

INFO: Converting image: ../data/raw\20220504/E14.czi
INFO: Overwrite disabled, so not saving ../data/interim/converted/images\20220504/E14/E14_000000.tif beacause it already exists
INFO: Converting image: ../data/raw\20220504/E14a.czi
INFO: Overwrite disabled, so not saving ../data/interim/converted/images\20220504/E14a/E14a_000000.tif beacause it already exists
INFO: Converting image: ../data/raw\20220504/E14e.czi
INFO: Overwrite disabled, so not saving ../data/interim/converted/images\20220504/E14e/E14e_000000.tif beacause it already exists
INFO: Converting image: ../data/raw\20220504/E14f.czi
INFO: Overwrite disabled, so not saving ../data/interim/converted/images\20220504/E14f/E14f_000000.tif beacause it already exists
INFO: Converting image: ../data/raw\20220504/E14h.czi
INFO: Overwrite disabled, so not saving ../data/interim/converted/images\20220504/E14h/E14h_000000.tif beacause it already exists
INFO: Converting image: ../data/raw\20220504/E4 b.czi
INFO: Overwrite disabled, so 